In [7]:
from google.adk.agents import SequentialAgent, LlmAgent
from google.adk.runners import InMemoryRunner
from dotenv import load_dotenv

# Load environment variables (GOOGLE_API_KEY)
load_dotenv()

GEMINI_MODEL = "gemini-2.0-flash"

# The first agent generates the initial draft.
generator = LlmAgent(
    name="DraftWriter",
    model=GEMINI_MODEL,  # Add model parameter
    description="Generates initial draft content on a given subject.",
    instruction="Write a short, informative paragraph about the user's subject.",
    output_key="draft_text" # The output is saved to this state key.
)

# The second agent critiques the draft from the first agent.
reviewer = LlmAgent(
    name="FactChecker",
    model=GEMINI_MODEL,  # Add model parameter
    description="Reviews a given text for factual accuracy and provides a structured critique.",
    instruction="""
    You are a meticulous fact-checker.
    1. Read the text provided in the state key 'draft_text'.
    2. Carefully verify the factual accuracy of all claims.
    3. Your final output must be a dictionary containing two keys:
       - "status": A string, either "ACCURATE" or "INACCURATE".
       - "reasoning": A string providing a clear explanation for your status, citing specific issues if any are found.
    """,
    output_key="review_output" # The structured dictionary is saved here.
)

# The SequentialAgent ensures the generator runs before the reviewer.
review_pipeline = SequentialAgent(
    name="WriteAndReview_Pipeline",
    sub_agents=[generator, reviewer]
)

# Execution Flow:
# 1. generator runs -> saves its paragraph to state['draft_text'].
# 2. reviewer runs -> reads state['draft_text'] and saves its dictionary output to state['review_output'].

In [8]:
print("=" * 80)
print("Starting Sequential pipeline")
print("=" * 80)
print("=" * 80)

runner = InMemoryRunner(review_pipeline)

# run the pipeline
result = await runner.run_debug("create content regarding the use of AI in the classroom", verbose=True)

print("\n" + "=" * 80)
print("Final Response:")
print("=" * 80)
print(result)
print("=" * 80)

Starting Sequential pipeline

 ### Created new session: debug_session_id

User > create content regarding the use of AI in the classroom
DraftWriter > Artificial intelligence (AI) is rapidly transforming the educational landscape, offering a range of tools and applications that can enhance teaching and learning. AI-powered platforms can personalize learning experiences by adapting to individual student needs and learning styles, providing customized content and feedback. Furthermore, AI can automate administrative tasks for teachers, freeing up their time to focus on student interaction and curriculum development. From AI-driven tutoring systems to automated grading and assessment tools, the integration of AI in the classroom holds the potential to improve student outcomes, increase efficiency, and foster a more engaging and effective learning environment.

FactChecker > ```json
{
  "status": "ACCURATE",
  "reasoning": "The text accurately describes the potential impact of AI in educat